## Scrapping points table from cricbuzz.com

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import os
import re
from IPython.display import clear_output
pd.set_option('display.max_colwidth', None)

In [ ]:
html_text=requests.get('https://m.cricbuzz.com/cricket-series/6732/icc-cricket-world-cup-2023/points-table')
print(html_text.status_code)
print(html_text.text)

200



<!DOCTYPE html><html lang="en" ng-app="CB"><head><meta charset="utf-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta name="viewport" content="width=device-width, initial-scale=1"><meta property="fb:app_id" content="30119633160" /><meta property="og:site_name" content="Cricbuzz" /><meta property="og:email" content="webmaster@cricbuzz.com" /><meta name="twitter:site" content="@cricbuzz" /><meta name="twitter:domain" content="cricbuzz.com" /><meta name="author" content=""><link rel="shortcut icon" type="image/x-icon" href="/cricket-series/images/cricbuzz/cb-favicon.ico" /><link rel="apple-touch-icon" href="/cricket-series/images/cricbuzz/cb_touch_icon.png" /><link rel="android-touch-icon" href="/cricket-series/images/cricbuzz/cb_touch_icon.png" /><title>ICC Cricket World Cup 2023 Points Table | Cricbuzz.com</title><meta name="description" content="Get a glimpse at the Points table of the ICC Cricket World Cup 2023 on Cricbuzz.com"><meta property="og:description" conten

In [ ]:
soup_text= BeautifulSoup("<p><li></p>",'lxml')
#print(soup_text)
soup=BeautifulSoup(html_text.text,'lxml')
soup

<!DOCTYPE html>
<html lang="en" ng-app="CB"><head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="30119633160" property="fb:app_id"/><meta content="Cricbuzz" property="og:site_name"/><meta content="webmaster@cricbuzz.com" property="og:email"/><meta content="@cricbuzz" name="twitter:site"/><meta content="cricbuzz.com" name="twitter:domain"/><meta content="" name="author"/><link href="/cricket-series/images/cricbuzz/cb-favicon.ico" rel="shortcut icon" type="image/x-icon"/><link href="/cricket-series/images/cricbuzz/cb_touch_icon.png" rel="apple-touch-icon"/><link href="/cricket-series/images/cricbuzz/cb_touch_icon.png" rel="android-touch-icon"/><title>ICC Cricket World Cup 2023 Points Table | Cricbuzz.com</title><meta content="Get a glimpse at the Points table of the ICC Cricket World Cup 2023 on Cricbuzz.com" name="description"/><meta content="Get a glimpse at the Points tabl

In [ ]:
# Initialize empty lists for data
countries = []
matches_played = []
matches_won = []
matches_lost = []
points = []
net_run_rate = []

content = soup.find('table', {'class': 'table'}).find_all('tbody')
for i in content:
    data = i.find_all('td')

    # Extract country name
    country_name = i.find('td', class_='team_name').text.strip()

    # Use regular expressions to remove "\xa0(E)" and "\xa0(Q)" and any non-breaking space characters
    cleaned_name = re.sub(r'\s*\(E\)', '', re.sub(r'\s*\(Q\)', '', country_name))
    countries.append(cleaned_name)

    # Extract M values (matches played, matches won, matches lost)
    m_played = data[1].text.strip()
    m_won = data[2].text.strip()
    m_lost = data[3].text.strip()
    matches_played.append(m_played)
    matches_won.append(m_won)
    matches_lost.append(m_lost)

    # Extract Pts and Nrr values
    pts_value = data[4].text.strip()
    nrr_value = data[5].text.strip()
    points.append(pts_value)
    net_run_rate.append(nrr_value)

countries, matches_played, matches_won, matches_lost, points, net_run_rate


(['IND', 'RSA', 'AUS', 'NZ', 'PAK', 'AFG', 'ENG', 'BAN', 'SL', 'NED'],
 ['9', '9', '9', '9', '9', '9', '9', '9', '9', '9'],
 ['9', '7', '7', '5', '4', '4', '3', '2', '2', '2'],
 ['0', '2', '2', '4', '5', '5', '6', '7', '7', '7'],
 ['18', '14', '14', '10', '8', '8', '6', '4', '4', '4'],
 ['+2.570',
  '+1.261',
  '+0.841',
  '+0.743',
  '-0.199',
  '-0.336',
  '-0.572',
  '-1.087',
  '-1.419',
  '-1.825'])

In [ ]:
df_pt=pd.DataFrame({'countries':countries, 'matches_played':matches_played, 'matches_won':matches_won, 'matches_lost':matches_lost,'points':points, 'net_run_rate':net_run_rate})
df_pt.to_csv('points_table.csv')

In [ ]:
df_pt

,countries,matches_played,matches_won,matches_lost,points,net_run_rate
0,IND,9,9,0,18,+2.570
1,RSA,9,7,2,14,+1.261
2,AUS,9,7,2,14,+0.841
3,NZ,9,5,4,10,+0.743
4,PAK,9,4,5,8,-0.199
5,AFG,9,4,5,8,-0.336
6,ENG,9,3,6,6,-0.572
7,BAN,9,2,7,4,-1.087
8,SL,9,2,7,4,-1.419
9,NED,9,2,7,4,-1.825


## Scrapping batting and bowling averages of each player for semifinalist teams from espncricinfo.com

In [ ]:
bat_IndHtml=requests.get('https://www.espncricinfo.com/records/tournament/averages-batting-bowling-by-team/icc-cricket-world-cup-2023-24-15338?team=6')
bat_AusHtml=requests.get('https://www.espncricinfo.com/records/tournament/averages-batting-bowling-by-team/icc-cricket-world-cup-2023-24-15338?team=2')
bat_NZHtml=requests.get('https://www.espncricinfo.com/records/tournament/averages-batting-bowling-by-team/icc-cricket-world-cup-2023-24-15338?team=5')
bat_SAHtml=requests.get('https://www.espncricinfo.com/records/tournament/averages-batting-bowling-by-team/icc-cricket-world-cup-2023-24-15338?team=3')


In [ ]:
def create_csv(html_content, country_code):
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('tbody')
    rows = table.find_all('tr')
    data_list = []

    for row in rows:
        columns = row.find_all(['td'])
        player_data = {
            'Player': columns[0].text.strip(),
            'Matches': columns[1].text.strip(),
            'Innings': columns[2].text.strip(),
            'Not Outs': columns[3].text.strip(),
            'Runs': columns[4].text.strip(),
            'BF' : columns[5].text.strip(),
            'Highest Score': columns[6].text.strip(),
            'Average': columns[7].text.strip(),
            'Strike Rate': columns[8].text.strip(),
            'Hundreds': columns[9].text.strip(),
            'Fifties': columns[10].text.strip(),
            'Ducks': columns[11].text.strip(),
            'Fours': columns[12].text.strip(),
            'Sixes': columns[13].text.strip(),
            'Country': country_code,
        }
        data_list.append(player_data)

    return data_list


# Create lists for each country
data_list_Ind = create_csv(bat_IndHtml.text, 'IND')
data_list_Aus = create_csv(bat_AusHtml.text, 'AUS')
data_list_NZ = create_csv(bat_NZHtml.text, 'NZ')
data_list_SA = create_csv(bat_SAHtml.text, 'SA')

# Combine data from all countries
combined_data_list = data_list_Ind + data_list_Aus + data_list_NZ + data_list_SA

# Write to a single CSV file
csv_file_path = 'combined_player_stats.csv'
keys = combined_data_list[0].keys()

with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=keys)
    writer.writeheader()
    writer.writerows(combined_data_list)


In [ ]:
from bs4 import BeautifulSoup

# Assume that batIND contains the HTML content
soup = BeautifulSoup(bat_IndHtml.text, 'html.parser')

# Find all tables with the specified class
tables = soup.find_all('table', class_='ds-w-full ds-table ds-table-xs ds-table-auto ds-w-full ds-overflow-scroll ds-scrollbar-hide')

# Check if there are at least two tables
if len(tables) >= 2:
    # Extract data from the second table
    second_table = tables[1]

    # Extract data from the second table
    rows = second_table.find_all('tr')

    table_data = []

    # Extract data from each row
    for row in rows:
        columns = row.find_all(['td', 'th'])
        player_data_ball = {
        'Player': columns[0].text.strip(),
        'Span': columns[1].text.strip(),
        'Mat': columns[2].text.strip(),
        'Inns': columns[3].text.strip(),
        'Overs': columns[4].text.strip(),
        'Mdns': columns[5].text.strip(),
        'Runs': columns[6].text.strip(),
        'Wkts': columns[7].text.strip(),
        'BBI': columns[8].text.strip(),
        'Ave': columns[9].text.strip(),
        'Econ': columns[10].text.strip(),
        'SR': columns[11].text.strip(),
        'Five': columns[12].text.strip(),
        'Ten': columns[13].text.strip(),
        'Ct': columns[14].text.strip(),
        'St': columns[15].text.strip(),
        'Country': 'IND',
    }
    table_data.append(player_data_ball)

    for data in table_data:
        print(data)


{'Player': 'KL Rahul', 'Span': '2023-2023', 'Mat': '9', 'Inns': '-', 'Overs': '0.0', 'Mdns': '-', 'Runs': '-', 'Wkts': '0', 'BBI': '-', 'Ave': '-', 'Econ': '-', 'SR': '-', 'Five': '-', 'Ten': '-', 'Ct': '11', 'St': '1', 'Country': 'IND'}


In [ ]:
def create_csv_forBall(html_content, country_code):
    # Assume that batIND contains the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    # Find all tables with the specified class
    tables = soup.find_all('table', class_='ds-w-full ds-table ds-table-xs ds-table-auto ds-w-full ds-overflow-scroll ds-scrollbar-hide')
    # Check if there are at least two tables
    if len(tables) >= 2:
        # Extract data from the second table
        second_table = tables[1]

        # Extract data from the second table
        rows = second_table.find_all('tr')

        table_data = []
        # Extract data from each row
        for row in rows:
            print(row)
            columns = row.find_all(['td', 'th'])
            player_data_ball = {
                'Player': columns[0].text.strip(),
                'Span': columns[1].text.strip(),
                'Mat': columns[2].text.strip(),
                'Inns': columns[3].text.strip(),
                'Overs': columns[4].text.strip(),
                'Mdns': columns[5].text.strip(),
                'Runs': columns[6].text.strip(),
                'Wkts': columns[7].text.strip(),
                'BBI': columns[8].text.strip(),
                'Ave': columns[9].text.strip(),
                'Econ': columns[10].text.strip(),
                'SR': columns[11].text.strip(),
                'Five': columns[12].text.strip(),
                'Ten': columns[13].text.strip(),
                'Ct': columns[14].text.strip(),
                'St': columns[15].text.strip(),
                'Country': country_code,
            }
            table_data.append(player_data_ball)

        return table_data

# Create lists for each country
ballData_list_Ind = create_csv_forBall(bat_IndHtml.text, 'IND')
ballData_list_Aus = create_csv_forBall(bat_AusHtml.text, 'AUS')
ballData_list_NZ = create_csv_forBall(bat_NZHtml.text, 'NZ')
ballData_list_SA = create_csv_forBall(bat_SAHtml.text, 'SA')

# Combine data from all countries
combined_data_list = ballData_list_Ind + ballData_list_Aus + ballData_list_NZ + ballData_list_SA

# Write to a single CSV file
csv_file_path = 'Ball_combined_player_stats.csv'
keys = combined_data_list[0].keys()

with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=keys)
    writer.writeheader()
    writer.writerows(combined_data_list)


## Scrapping player details from espncricinfo.com

In [ ]:
mostRuns=requests.get('https://www.espncricinfo.com/records/tournament/batting-most-runs-career/icc-cricket-world-cup-2023-24-15338')
mostWicktes=requests.get('https://www.espncricinfo.com/records/tournament/bowling-most-wickets-career/icc-cricket-world-cup-2023-24-15338')
mostCaches=requests.get('https://www.espncricinfo.com/records/tournament/fielding-most-catches-career/icc-cricket-world-cup-2023-24-15338')

In [ ]:
# Create a BeautifulSoup object
soup = BeautifulSoup(mostRuns.text, 'html.parser')

# Find the table containing player data
table = soup.find('table', class_='ds-table')

# Check if the table is found
if table:
    # Find all rows in the table
    rows = table.find_all('tr')

    # Lists to store data
    player_names = []
    country_codes = []
    spans = []
    matches = []
    innings = []
    not_outs = []
    runs = []
    highest_scores = []
    averages = []
    balls_faced = []
    strike_rates = []
    hundreds = []
    fifties = []
    ducks = []
    fours = []
    sixes = []

    # Extract data from each row
    for row in rows[1:]:  # Skip the header row
        columns = row.find_all('td')

        # Extract data for each column
        player_name = columns[0].text.strip()
        # Extract country code from the player name within brackets
        country_code = player_name.split('(')[-1].strip(')') if '(' in player_name else ''
        country_codes.append(country_code)
        player_names.append(player_name.replace("("+country_code+")",'').strip())
        spans.append(columns[1].text.strip())
        matches.append(columns[2].text.strip())
        innings.append(columns[3].text.strip())
        not_outs.append(columns[4].text.strip())
        runs.append(columns[5].text.strip())
        highest_scores.append(columns[6].text.strip())
        averages.append(columns[7].text.strip())
        balls_faced.append(columns[8].text.strip())
        strike_rates.append(columns[9].text.strip())
        hundreds.append(columns[10].text.strip())
        fifties.append(columns[11].text.strip())
        ducks.append(columns[12].text.strip())
        fours.append(columns[13].text.strip())
        sixes.append(columns[14].text.strip())

    # Write data to CSV file
    with open('Most_Runs.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # Write header
        writer.writerow(['Player','Country', 'Span', 'Matches', 'Innings', 'Not Outs', 'Runs', 'Highest Score', 'Average', 'Balls Faced', 'Strike Rate', 'Hundreds', 'Fifties', 'Zeros', 'Fours', 'Sixes'])
        # Write data
        for i in range(len(player_names)):
            writer.writerow([player_names[i], country_codes[i],spans[i], matches[i], innings[i], not_outs[i], runs[i], highest_scores[i], averages[i], balls_faced[i], strike_rates[i], hundreds[i], fifties[i], ducks[i], fours[i], sixes[i]])

    print("CSV file created successfully.")
else:
    print("Table not found.")


CSV file created successfully.


In [ ]:
#MOST Wickets
# Create a BeautifulSoup object
soup = BeautifulSoup(mostWicktes.text, 'html.parser')

# Find the table containing player data
table = soup.find('table', class_='ds-table')

# Check if the table is found
if table:
    # Find all rows in the table
    rows = table.find_all('tr')

    # Lists to store data
    player_names = []
    country_codes = []
    spans = []
    matches = []
    innings = []
    not_outs = []
    runs = []
    highest_scores = []
    averages = []
    balls_faced = []
    strike_rates = []
    hundreds = []
    fifties = []
    ducks = []
    fours = []
    sixes = []

    # Extract data from each row
    for row in rows[1:]:  # Skip the header row
        columns = row.find_all('td')

        # Extract data for each column
        player_name = columns[0].text.strip()

        # Extract country code from the player name within brackets
        country_code = player_name.split('(')[-1].strip(')') if '(' in player_name else ''
        country_codes.append(country_code)
        player_names.append(player_name.replace("("+country_code+")",'').strip())
        spans.append(columns[1].text.strip())
        matches.append(columns[2].text.strip())
        innings.append(columns[3].text.strip())
        not_outs.append(columns[4].text.strip())
        runs.append(columns[5].text.strip())
        highest_scores.append(columns[6].text.strip())
        averages.append(columns[7].text.strip())
        balls_faced.append(columns[8].text.strip())
        strike_rates.append(columns[9].text.strip())
        hundreds.append(columns[10].text.strip())
        fifties.append(columns[11].text.strip())
        ducks.append(columns[12].text.strip())
        fours.append(columns[13].text.strip())
        sixes.append(columns[14].text.strip())

    # Write data to CSV file
    with open('most_wickets.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # Write header
        writer.writerow(['Player', 'Country', 'Span', 'Matches', 'Innings', 'Balls', 'Overs', 'MDNS', 'Runs', 'Wkts', 'BBI', 'AVE', 'Econ', 'SR', 'Fours', 'Fives'])
        # Write data
        for i in range(len(player_names)):
            writer.writerow([player_names[i], country_codes[i], spans[i], matches[i], innings[i], not_outs[i], runs[i], highest_scores[i], averages[i], balls_faced[i], strike_rates[i], hundreds[i], fifties[i], ducks[i], fours[i], sixes[i]])

    print("CSV file created successfully.")
else:
    print("Table not found.")


In [ ]:
from bs4 import BeautifulSoup
import csv

# Create a BeautifulSoup object
soup = BeautifulSoup(mostCaches.text, 'html.parser')

# Find the table containing player data
table = soup.find('table', class_='ds-table')

# Check if the table is found
if table:
    # Find all rows in the table
    rows = table.find_all('tr')

    # Lists to store data
    player_names = []
    country_codes = []
    spans = []
    matches = []
    innings = []
    CT = []
    Max = []
    Ct_Inns =   []

    # Extract data from each row
    for row in rows[1:]:  # Skip the header row
        columns = row.find_all('td')

        # Extract data for each column
        player_name = columns[0].text.strip()

        # Extract country code from the player name within brackets
        country_code = player_name.split('(')[-1].strip(')') if '(' in player_name else ''
        country_codes.append(country_code)
        player_names.append(player_name.replace("("+country_code+")",'').strip())

        spans.append(columns[1].text.strip())
        matches.append(columns[2].text.strip())
        innings.append(columns[3].text.strip())
        CT.append(columns[4].text.strip())
        Max.append(columns[5].text.strip())
        Ct_Inns.append(columns[6].text.strip())

    # Write data to CSV file
    with open('most_catches.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # Write header
        writer.writerow(['Player', 'Country', 'Span', 'Matches', 'Innings', 'CT', 'Max', 'Ct/Inns'])
        # Write data
        for i in range(len(player_names)):
            writer.writerow([player_names[i], country_codes[i], spans[i], matches[i], innings[i], CT[i], Max[i], Ct_Inns[i]])

    print("CSV file created successfully.")
else:
    print("Table not found.")
